In [1]:
#!pip install holidays

In [2]:
# Initial imports to get show started
import datetime as dt
import holidays
import pandas as pd
from pathlib import Path

In [3]:
# Read in BTC historical hourly price data 

# data downloaded from CryptoDataDownload with Date column natively in NY EST zone
btc_df = pd.read_csv(Path('./RawData/gemini_BTCUSD_1hr.csv'),parse_dates=['Date'])
btc_df.tail(3)

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
46206,1.444316e+09,2015-10-08 15:00:00,BTCUSD,245.0,245.0,244.92,244.92,3.016926
46207,1.444313e+09,2015-10-08 14:00:00,BTCUSD,245.0,245.0,244.50,245.00,4.453649
46208,1.444309e+09,2015-10-08 13:00:00,BTCUSD,0.0,245.0,0.00,245.00,0.606654


In [4]:
# Clean and prep dataframe for model use

# remove extra unused columns
btc_df = btc_df.drop(columns=["Unix Timestamp","Symbol","Open","High","Low"])

# rename Date column and create new Date-only column of string type
btc_df.rename(columns=({"Date":"Date_Time"}), inplace=True)
btc_df['Date'] = btc_df['Date_Time'].dt.strftime('%Y/%m/%d')

# create Weekday column of int type from renamed Date_Time
btc_df['Weekday'] = btc_df['Date_Time'].dt.dayofweek

# create Hour column of int type from renamed Date_Time
btc_df['Hour'] = btc_df['Date_Time'].dt.strftime('%H').astype(int)

# set renamed Date_Time as index and arrange dates in ascending order
btc_df.set_index(['Date_Time'],inplace=True)
btc_df = btc_df.sort_values(by='Date_Time',ascending=True)

btc_df.tail(3)

,Close,Volume,Date,Weekday,Hour
Date_Time,,,,,
2021-01-14 22:00:00,38981.41,92.893830,2021/01/14,3,22
2021-01-14 23:00:00,39147.75,57.506068,2021/01/14,3,23
2021-01-15 00:00:00,39443.67,9.715842,2021/01/15,4,0


In [5]:
# Verify datatypes dealing with
print(btc_df.dtypes)

Close      float64
Volume     float64
Date        object
Weekday      int64
Hour         int32
dtype: object


In [6]:
# Calculate Date/Time based model features

# create US_Holiday flag from Date column
us_holidays = []
for date in holidays.UnitedStates(years=[2015,2016,2017,2018,2019,2020,2021]).items():
    us_holidays.append(str(date[0]))

btc_df['US_Holiday'] = [1 if day in us_holidays else 0 for day in btc_df['Date']]

# create US_Market_Open flag from Hour, Weekday, US_Holiday columns
#  pythonic Monday = 0, assume institutional capital active M-F 9am-5pm
btc_df['US_Market_Open'] = [1 if ((day<=5)and(hour>=9)and(hour<17)and(holiday!=1)) else 0
                            for day,hour,holiday
                            in zip(btc_df['Weekday'],btc_df['Hour'],btc_df['US_Holiday'])]
btc_df.head(10)

,Close,Volume,Date,Weekday,Hour,US_Holiday,US_Market_Open
Date_Time,,,,,,,
2015-10-08 13:00:00,245.00,0.606654,2015/10/08,3,13,0,1
2015-10-08 14:00:00,245.00,4.453649,2015/10/08,3,14,0,1
2015-10-08 15:00:00,244.92,3.016926,2015/10/08,3,15,0,1
2015-10-08 16:00:00,244.25,3.895252,2015/10/08,3,16,0,1
2015-10-08 17:00:00,244.99,3.920632,2015/10/08,3,17,0,0
2015-10-08 18:00:00,244.00,3.690472,2015/10/08,3,18,0,0
2015-10-08 19:00:00,244.00,1.531456,2015/10/08,3,19,0,0
2015-10-08 20:00:00,244.00,0.177049,2015/10/08,3,20,0,0
2015-10-08 21:00:00,243.95,0.041322,2015/10/08,3,21,0,0


In [7]:
# Calculate price derived features for model

# define hours per window size since dataset is hourly price data
one_day = 24
twelve_week = 2016
twelve_month = 8736

# calculate price ratio to all time and rolling window maximums
all_time_max_price = btc_df.Close.max() 
btc_df['Trail24hr_MaxClose'] = btc_df.Close.rolling(one_day).max()
btc_df['Trail12Wk_MaxClose'] = btc_df.Close.rolling(twelve_week).max()
btc_df['Trail52Wk_MaxClose'] = btc_df.Close.rolling(twelve_month).max()

#btc_df['TrailAll_CloseRatio'] = [(btc_df.Close)/(all_time_max_price)]
btc_df['Trail24hr_CloseRatio'] = btc_df.Close / btc_df.Trail24hr_MaxClose
btc_df['Trail12Wk_CloseRatio'] = btc_df.Close / btc_df.Trail12Wk_MaxClose
btc_df['Trail52Wk_CloseRatio'] = btc_df.Close / btc_df.Trail52Wk_MaxClose

# calculate price percent change per hour
btc_df['Hr_Return'] = btc_df.Close.pct_change()

# calculate price percent change, std per sliding 24hr period
btc_df['Trail24hr_Return'] = btc_df.Hr_Return.rolling(one_day).sum()
btc_df['Trail24hr_Std'] = btc_df.Hr_Return.rolling(one_day).std()

# calculate price percent change, std per sliding twelve weeks period
btc_df['Trail12Wk_Return'] = btc_df.Hr_Return.rolling(twelve_week).sum()
btc_df['Trail12Wk_Std'] = btc_df.Hr_Return.rolling(twelve_week).std()

# calculate price percent change, std per sliding 52 week period
btc_df['Trail52Wk_Return'] = btc_df.Hr_Return.rolling(twelve_month).sum()
btc_df['Trail52Wk_Std'] = btc_df.Hr_Return.rolling(twelve_month).std()

btc_df.tail()

,Close,Volume,Date,Weekday,Hour,US_Holiday,US_Market_Open,Trail24hr_MaxClose,Trail12Wk_MaxClose,Trail52Wk_MaxClose,Trail24hr_CloseRatio,Trail12Wk_CloseRatio,Trail52Wk_CloseRatio,Hr_Return,Trail24hr_Return,Trail24hr_Std,Trail12Wk_Return,Trail12Wk_Std,Trail52Wk_Return,Trail52Wk_Std
Date_Time,,,,,,,,,,,,,,,,,,,,
2021-01-14 20:00:00,39336.89,52.909791,2021/01/14,3,20,0,0,39885.61,41526.95,41526.95,0.986243,0.947262,0.947262,-0.013757,0.082651,0.011074,1.184965,0.009269,1.830268,0.008611
2021-01-14 21:00:00,38773.68,181.618546,2021/01/14,3,21,0,0,39885.61,41526.95,41526.95,0.972122,0.933699,0.933699,-0.014318,0.040132,0.010316,1.167364,0.009275,1.822966,0.008612
2021-01-14 22:00:00,38981.41,92.893830,2021/01/14,3,22,0,0,39885.61,41526.95,41526.95,0.977330,0.938701,0.938701,0.005358,0.046768,0.010322,1.183481,0.009272,1.824885,0.008613
2021-01-14 23:00:00,39147.75,57.506068,2021/01/14,3,23,0,0,39885.61,41526.95,41526.95,0.981501,0.942707,0.942707,0.004267,0.046392,0.010318,1.190487,0.009272,1.828135,0.008613
2021-01-15 00:00:00,39443.67,9.715842,2021/01/15,4,0,0,0,39885.61,41526.95,41526.95,0.988920,0.949833,0.949833,0.007559,0.042761,0.010203,1.202354,0.009272,1.839694,0.008613


In [8]:
# Calculate volume derived features for model

# TODO There is a bug believed related to zero values that causes erroneous pct changes!!

# calculate volume ratio to rolling window maximums
btc_df['Trail24hr_MaxVol'] = btc_df.Volume.rolling(one_day).max()
btc_df['Trail12Wk_MaxVol'] = btc_df.Volume.rolling(twelve_week).max()
btc_df['Trail52Wk_MaxVol'] = btc_df.Volume.rolling(twelve_month).max()

btc_df['Trail24hr_VolRatio'] = btc_df.Volume / btc_df.Trail24hr_MaxVol
btc_df['Trail12Wk_VolRatio'] = btc_df.Volume / btc_df.Trail12Wk_MaxVol
btc_df['Trail52Wk_VolRatio'] = btc_df.Volume / btc_df.Trail52Wk_MaxVol

# calculate volume percent change per hour
btc_df['Vol_PctDelta'] = btc_df.Volume.pct_change()

btc_df.tail()

,Close,Volume,Date,Weekday,Hour,US_Holiday,US_Market_Open,Trail24hr_MaxClose,Trail12Wk_MaxClose,Trail52Wk_MaxClose,...,Trail12Wk_Std,Trail52Wk_Return,Trail52Wk_Std,Trail24hr_MaxVol,Trail12Wk_MaxVol,Trail52Wk_MaxVol,Trail24hr_VolRatio,Trail12Wk_VolRatio,Trail52Wk_VolRatio,Vol_PctDelta
Date_Time,,,,,,,,,,,,,,,,,,,,,
2021-01-14 20:00:00,39336.89,52.909791,2021/01/14,3,20,0,0,39885.61,41526.95,41526.95,...,0.009269,1.830268,0.008611,375.83964,1696.47052,3135.605196,0.140778,0.031188,0.016874,-0.584806
2021-01-14 21:00:00,38773.68,181.618546,2021/01/14,3,21,0,0,39885.61,41526.95,41526.95,...,0.009275,1.822966,0.008612,375.83964,1696.47052,3135.605196,0.483234,0.107057,0.057921,2.432607
2021-01-14 22:00:00,38981.41,92.893830,2021/01/14,3,22,0,0,39885.61,41526.95,41526.95,...,0.009272,1.824885,0.008613,375.83964,1696.47052,3135.605196,0.247163,0.054757,0.029625,-0.488522
2021-01-14 23:00:00,39147.75,57.506068,2021/01/14,3,23,0,0,39885.61,41526.95,41526.95,...,0.009272,1.828135,0.008613,375.83964,1696.47052,3135.605196,0.153007,0.033897,0.018340,-0.380948
2021-01-15 00:00:00,39443.67,9.715842,2021/01/15,4,0,0,0,39885.61,41526.95,41526.95,...,0.009272,1.839694,0.008613,375.83964,1696.47052,3135.605196,0.025851,0.005727,0.003099,-0.831047


In [9]:
# Generate training target from 24hr window on Returns

drawdown_threshold_pct = 0.20
btc_df['Significant_Drawdown'] = [1 if percent < -drawdown_threshold_pct else 0
                                    for percent
                                    in btc_df.Trail24hr_Return]

In [10]:
# Remove all undefined rows (52Weeks worth) on BTC dataset calculations
btc_df = btc_df.dropna()

In [11]:
# Merge BTC df with social media signal dataset to create final model df
model_df = btc_df.copy()
model_df.head()

,Close,Volume,Date,Weekday,Hour,US_Holiday,US_Market_Open,Trail24hr_MaxClose,Trail12Wk_MaxClose,Trail52Wk_MaxClose,...,Trail52Wk_Return,Trail52Wk_Std,Trail24hr_MaxVol,Trail12Wk_MaxVol,Trail52Wk_MaxVol,Trail24hr_VolRatio,Trail12Wk_VolRatio,Trail52Wk_VolRatio,Vol_PctDelta,Significant_Drawdown
Date_Time,,,,,,,,,,,,,,,,,,,,,
2016-10-06 13:00:00,612.29,142.121022,2016/10/06,3,13,0,1,613.18,684.93,788.55,...,1.178885,0.007725,1817.560457,3176.491887,3176.491887,0.078193,0.044742,0.044742,3.744561,0
2016-10-06 14:00:00,612.99,139.477101,2016/10/06,3,14,0,1,613.10,684.93,788.55,...,1.180028,0.007725,1817.560457,3176.491887,3176.491887,0.076739,0.043909,0.043909,-0.018603,0
2016-10-06 15:00:00,613.18,57.461061,2016/10/06,3,15,0,1,613.18,684.93,788.55,...,1.180665,0.007725,1817.560457,3176.491887,3176.491887,0.031614,0.018089,0.018089,-0.588025,0
2016-10-06 16:00:00,612.65,56.431175,2016/10/06,3,16,0,1,613.18,684.93,788.55,...,1.182536,0.007725,1817.560457,3176.491887,3176.491887,0.031048,0.017765,0.017765,-0.017923,0
2016-10-06 17:00:00,612.27,98.703355,2016/10/06,3,17,0,0,613.18,684.93,788.55,...,1.178886,0.007725,1817.560457,3176.491887,3176.491887,0.054305,0.031073,0.031073,0.749093,0


In [12]:
# Print stats on final model_dataset

num_significant_drawdown = model_df.Significant_Drawdown.sum()
print(f"Number of Significant Drawdowns > {drawdown_threshold_pct*100}% is {num_significant_drawdown}!")

model_df.info()

Number of Significant Drawdowns > 20.0% is 56!
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 37337 entries, 2016-10-06 13:00:00 to 2021-01-15 00:00:00
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Close                 37337 non-null  float64
 1   Volume                37337 non-null  float64
 2   Date                  37337 non-null  object 
 3   Weekday               37337 non-null  int64  
 4   Hour                  37337 non-null  int32  
 5   US_Holiday            37337 non-null  int64  
 6   US_Market_Open        37337 non-null  int64  
 7   Trail24hr_MaxClose    37337 non-null  float64
 8   Trail12Wk_MaxClose    37337 non-null  float64
 9   Trail52Wk_MaxClose    37337 non-null  float64
 10  Trail24hr_CloseRatio  37337 non-null  float64
 11  Trail12Wk_CloseRatio  37337 non-null  float64
 12  Trail52Wk_CloseRatio  37337 non-null  float64
 13  Hr_Return             37337 non-null  f

In [13]:
# Export merged model_dataset to csv, exporting only feature columns to be utilized
model_col_list = ['Close',
                  'Volume',
                  'US_Holiday',
                  'US_Market_Open',
                  'Trail24hr_CloseRatio',
                  'Trail12Wk_CloseRatio',
                  'Trail52Wk_CloseRatio',
                  'Hr_Return',
                  'Trail24hr_Return',
                  'Trail24hr_Std',
                  'Trail12Wk_Return',
                  'Trail12Wk_Std',
                  'Trail52Wk_Return',
                  'Trail52Wk_Std',
                  'Trail24hr_VolRatio',
                  'Trail12Wk_VolRatio',
                  'Trail52Wk_VolRatio',
                  'Vol_PctDelta',
                  'Significant_Drawdown']

model_df.to_csv('./ModelData/model_dataset.csv',columns=model_col_list)